In [ ]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np
import time
import warnings
import tkintermapview
from ultralytics import YOLO
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.linear_model import RANSACRegressor  # Added for robust fitting
import pyttsx3  # Text-to-speech

# Initialize the TTS engine
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)  # Slower speech rate
voices = tts_engine.getProperty('voices')
for idx, voice in enumerate(voices):
    print(f"Voice {idx}: {voice.name}")
tts_engine.setProperty('voice', voices[0].id)  # Choose a voice
tts_engine.setProperty('volume', 0.9)

def speak_message(message):
    tts_engine.say(message)
    tts_engine.runAndWait()

try:
    from numpy import RankWarning
except ImportError:
    class RankWarning(Warning):
        pass
warnings.simplefilter('ignore', RankWarning)

# --------------------- Enhanced Lane Detection Classes ---------------------
class LaneSmoother:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.left_fit = None
        self.right_fit = None

    def smooth(self, left_fit, right_fit):
        if left_fit is None or right_fit is None:
            self.left_fit = left_fit
            self.right_fit = right_fit
        else:
            if self.left_fit is None:
                self.left_fit = left_fit
            else:
                self.left_fit = self.alpha * left_fit + (1 - self.alpha) * self.left_fit
            if self.right_fit is None:
                self.right_fit = right_fit
            else:
                self.right_fit = self.alpha * right_fit + (1 - self.alpha) * self.right_fit
        return self.left_fit, self.right_fit

class ModeSwitcher:
    def __init__(self, curvature_threshold=0.0003, straight_time=2.0, curve_time=1.0):
        self.mode = "curved"
        self.curvature_threshold = curvature_threshold
        self.straight_time = straight_time
        self.curve_time = curve_time
        self.straight_start = None
        self.curve_start = None

    def update(self, left_fit, right_fit):
        current_time = time.time()
        coeffs = []
        if left_fit is not None:
            coeffs.append(abs(left_fit[0]))
        if right_fit is not None:
            coeffs.append(abs(right_fit[0]))
        if not coeffs:
            return self.mode
        avg_curvature = np.mean(coeffs)
        if self.mode == "curved":
            if avg_curvature < self.curvature_threshold:
                if self.straight_start is None:
                    self.straight_start = current_time
                elif current_time - self.straight_start >= self.straight_time:
                    self.mode = "straight"
                    self.straight_start = None
            else:
                self.straight_start = None
        else:
            if avg_curvature >= self.curvature_threshold:
                if self.curve_start is None:
                    self.curve_start = current_time
                elif current_time - self.curve_start >= self.curve_time:
                    self.mode = "curved"
                    self.curve_start = None
            else:
                self.curve_start = None
        return self.mode

# --------------------- New Lane Detection Functions (Bird’s-Eye View & Sliding Window) ---------------------
def undistort_image(image):
    # In a full implementation you may want to calibrate your camera.
    # Here, we assume the image is undistorted.
    return image

def get_perspective_transform(image):
    height, width = image.shape[:2]
    # Define source points (assumes a trapezoid covering the lane area)
    src = np.float32([
        [width * 0.45, height * 0.65],
        [width * 0.55, height * 0.65],
        [width * 0.9, height],
        [width * 0.1, height]
    ])
    # Define destination points for a bird's-eye view
    dst = np.float32([
        [width * 0.2, 0],
        [width * 0.8, 0],
        [width * 0.8, height],
        [width * 0.2, height]
    ])
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

def warp_image(image, M):
    height, width = image.shape[:2]
    warped = cv2.warpPerspective(image, M, (width, height), flags=cv2.INTER_LINEAR)
    return warped

def preprocess_lane_image(image):
    # Convert to HLS and threshold on white and yellow
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    lower_white = np.array([0, 200, 0])
    upper_white = np.array([255, 255, 255])
    white_mask = cv2.inRange(hls, lower_white, upper_white)
    
    lower_yellow = np.array([15, 30, 115])
    upper_yellow = np.array([35, 204, 255])
    yellow_mask = cv2.inRange(hls, lower_yellow, upper_yellow)
    
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    result = cv2.bitwise_and(image, image, mask=mask)
    gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    ret, binary = cv2.threshold(blur, 160, 255, cv2.THRESH_BINARY)
    return binary

def sliding_window_lane_search(binary_warped):
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:, :], axis=0)
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    midpoint = int(histogram.shape[0] / 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    nwindows = 9
    window_height = int(binary_warped.shape[0] / nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    minpix = 50

    leftx_current = leftx_base
    rightx_current = rightx_base

    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window + 1) * window_height
        win_y_high = binary_warped.shape[0] - window * window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) &
                           (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))
    
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fit = None
    right_fit = None
    if len(leftx) > 0:
        left_fit = np.polyfit(lefty, leftx, 2)
    if len(rightx) > 0:
        right_fit = np.polyfit(righty, rightx, 2)
        
    return left_fit, right_fit, out_img

def draw_lane_overlay(original_img, binary_warped, left_fit, right_fit, Minv):
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    color_warp = np.zeros_like(original_img).astype(np.uint8)
    
    if left_fit is not None:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    else:
        left_fitx = None
    if right_fit is not None:
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    else:
        right_fitx = None

    if left_fitx is not None and right_fitx is not None:
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
    newwarp = cv2.warpPerspective(color_warp, Minv, (original_img.shape[1], original_img.shape[0])) 
    result = cv2.addWeighted(original_img, 1, newwarp, 0.3, 0)
    return result

def detect_lanes(frame, smoother, mode_switcher):
    undistorted = undistort_image(frame)
    M, Minv = get_perspective_transform(undistorted)
    warped = warp_image(undistorted, M)
    binary_warped = preprocess_lane_image(warped)
    left_fit, right_fit, _ = sliding_window_lane_search(binary_warped)
    
    # Optionally, update mode based on curvature (using left_fit coefficients)
    current_mode = mode_switcher.update(left_fit, right_fit)
    # Smooth the lane fits
    if left_fit is not None and right_fit is not None:
        left_fit, right_fit = smoother.smooth(left_fit, right_fit)
        
    final_frame = draw_lane_overlay(frame, binary_warped, left_fit, right_fit, Minv)
    return left_fit, right_fit, current_mode, final_frame

# ---------------- Lane Departure Warning Logic ----------------
lane_warning_spoken = False  # Global flag to repeat the warning only once

def lane_departure_warning(left_fit, right_fit, frame_width, y_eval):
    global lane_warning_spoken
    threshold = 50  # Pixel threshold for departure
    if left_fit is None or right_fit is None:
        lane_warning_spoken = False
        return None
    left_x = np.polyval(left_fit, y_eval)
    right_x = np.polyval(right_fit, y_eval)
    lane_center = (left_x + right_x) / 2.0
    vehicle_center = frame_width / 2.0
    offset = vehicle_center - lane_center
    if offset < -threshold:  # Vehicle is too far left
        if not lane_warning_spoken:
            lane_warning_spoken = True
            return "Lane departure warning! Your vehicle is turning left. Regain balance."
        else:
            return None
    elif offset > threshold:  # Vehicle is too far right
        if not lane_warning_spoken:
            lane_warning_spoken = True
            return "Lane departure warning! Your vehicle is turning right. Regain balance."
        else:
            return None
    else:
        lane_warning_spoken = False
        return None

# ---------------- Object Detection ----------------
def detect_boxes(frame):
    global yolo_model, dev_log_messages
    pre_start = time.time()
    pre_time = (time.time() - pre_start) * 1000
    inf_start = time.time()
    results = yolo_model(frame)
    inf_time = (time.time() - inf_start) * 1000
    post_start = time.time()
    counts = {}
    for result in results:
        for box in result.boxes:
            cls = int(box.cls[0])
            counts[cls] = counts.get(cls, 0) + 1
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            label = yolo_model.names.get(cls, str(cls))
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
    post_time = (time.time() - post_start) * 1000
    detection_details = []
    for cls, count in counts.items():
        class_name = yolo_model.names.get(cls, str(cls))
        detection_details.append(f"{count} {class_name}")
    detail_str = ", ".join(detection_details)
    log_str = (f"0: {frame.shape[0]}x{frame.shape[1]} {detail_str}, {inf_time:.1f}ms\n"
               f"Speed: {pre_time:.1f}ms preprocess, {inf_time:.1f}ms inference, {post_time:.1f}ms postprocess per image at shape (1, 3, {frame.shape[0]}, {frame.shape[1]})")
    if is_developer_mode_active:
        dev_log_messages.append(log_str)
        if len(dev_log_messages) > 10:
            dev_log_messages[:] = dev_log_messages[-10:]
        update_dev_log_widget()
    return frame, counts

# ---------------- Developer Mode Data and Plotting ----------------
dev_times = []
dev_lane_indicator = []      # 1 if lanes detected, else 0
dev_person_counts = []
dev_car_counts = []
dev_motorcycle_counts = []
dev_other_counts = {}
dev_log_messages = []

def update_developer_plots():
    global dev_fig, dev_canvas, dev_times, dev_lane_indicator, dev_person_counts, dev_car_counts, dev_motorcycle_counts
    if dev_fig is None:
        return
    dev_fig.clf()
    ax1 = dev_fig.add_subplot(211)
    ax2 = dev_fig.add_subplot(212)
    if dev_times:
        ax1.plot(dev_times, dev_lane_indicator, label="Lane Detected (1/0)", color='green')
        ax1.set_ylim(-0.1, 1.1)
        ax1.set_title("Lane Detection Indicator")
        ax1.legend(fontsize=8)
        ax2.plot(dev_times, dev_person_counts, label="Persons", color='blue')
        ax2.plot(dev_times, dev_car_counts, label="Cars", color='orange')
        ax2.plot(dev_times, dev_motorcycle_counts, label="Motorcycles", color='red')
        ax2.set_title("Object Detection Counts")
        ax2.legend(fontsize=8)
    dev_fig.tight_layout()
    dev_canvas.draw()

def limit_dev_data(max_points=50):
    global dev_times, dev_lane_indicator, dev_person_counts, dev_car_counts, dev_motorcycle_counts
    if len(dev_times) > max_points:
        dev_times = dev_times[-max_points:]
        dev_lane_indicator = dev_lane_indicator[-max_points:]
        dev_person_counts = dev_person_counts[-max_points:]
        dev_car_counts = dev_car_counts[-max_points:]
        dev_motorcycle_counts = dev_motorcycle_counts[-max_points:]

def update_dev_log_widget():
    global dev_log_widget, dev_log_messages
    dev_log_widget.config(state=NORMAL)
    dev_log_widget.delete("1.0", END)
    dev_log_widget.insert(END, "\n".join(dev_log_messages))
    dev_log_widget.config(state=DISABLED)

def update_mode_info():
    if is_developer_mode_active:
        info_text = ""
        if is_lane_detection_active:
            info_text += ("In lane detection mode, the system identifies lane markings "
                          "using a bird’s-eye view and sliding window search.\n\n")
        if is_box_detection_active:
            info_text += ("In object detection mode, the program is trying to detect the type and "
                          "the quantity of traffic in front of the vehicle using YOLOv8.\n\n")
        if is_driver_assistance_active:
            info_text += ("In driver assistance mode, lane detection and object detection are integrated "
                          "to provide comprehensive feedback.\n\n")
        info_text += (f"\n[Status] Lane Assistance: {'ON' if is_lane_detection_active else 'OFF'} | "
                      f"Object Detection: {'ON' if is_box_detection_active else 'OFF'} | "
                      f"Driver Assistance: {'ON' if is_driver_assistance_active else 'OFF'}")
        mode_info_label.config(text=info_text)
    root.after(500, update_mode_info)

# ---------------- Global Variables and Mode Flags ----------------
vid = None
camera_running = False
is_lane_detection_active = False
is_box_detection_active = False
is_driver_assistance_active = False
is_developer_mode_active = False
is_learner_mode_active = False

# Global counters for learner mode instructions (only repeat each instruction three times)
person_warning_count = 0
traffic_warning_count = 0
traffic_light_warning_count = 0

lane_smoother = LaneSmoother(alpha=0.1)
mode_switcher = ModeSwitcher(curvature_threshold=0.001, straight_time=1.5, curve_time=1.5)

yolo_model = YOLO("yolov8n.pt")

# ---------------- Tkinter GUI Setup ----------------
root = Tk()
root.title("Advanced Self Driving")
root.geometry("1100x600")
root.resizable(0, 0)

background_image = PhotoImage(file="background.png")
background_label = Label(root, image=background_image)
background_label.place(x=0, y=0, relwidth=1, relheight=1)
Title_label = Label(root, text="Advanced\nSelf Driving", font="Elephant 18 bold", bg='#0f6e6e', fg='white')
Title_label.place(x=90, y=30)
app_icon = PhotoImage(file="road.png")
root.iconphoto(False, app_icon)
app_logo = PhotoImage(file="smart-car.png")
app_logo_resize = app_logo.subsample(8, 8)
app_logo_label = Label(root, image=app_logo_resize, bg='#0f6e6e')
app_logo_label.place(x=253, y=30)
camera_module = PhotoImage(file="camera_module.png")
camera_module_resize = camera_module.subsample(2, 2)
camera_module_label = Label(root, image=camera_module_resize, bg='#c5ece9', border=3)
camera_module_label.place(x=403, y=100)

# ---------------- Developer Mode Panels (separated) ----------------
dev_frame_left = Frame(root, bg='#c5ece9', bd=2, relief=SUNKEN)
dev_frame_right = Frame(root, bg='#c5ece9', bd=2, relief=SUNKEN)
dev_frame_left.place_forget()
dev_frame_right.place_forget()

dev_fig = plt.Figure(figsize=(3, 2), dpi=80)
dev_canvas = FigureCanvasTkAgg(dev_fig, master=dev_frame_left)
dev_canvas.get_tk_widget().pack(side=TOP, fill=BOTH, expand=True)

dev_frame_right.grid_rowconfigure(0, weight=1)
dev_frame_right.grid_rowconfigure(1, weight=1)
dev_frame_right.grid_columnconfigure(0, weight=1)
map_widget = tkintermapview.TkinterMapView(dev_frame_right, corner_radius=0)
map_widget.set_position(17.4065, 78.4772)
map_widget.set_zoom(10)
map_widget.grid(row=0, column=0, sticky="nsew")
dev_log_widget = Text(dev_frame_right, state=DISABLED, font=("Consolas", 8))
dev_log_widget.grid(row=1, column=0, sticky="nsew")

mode_info_frame = Frame(root, bg='#c5ece9', bd=2, relief=SUNKEN)
mode_info_label = Label(mode_info_frame, text="", font=("Arial", 10), bg='#c5ece9', wraplength=280, justify=CENTER)
mode_info_label.pack(fill=BOTH, expand=True)

fps_label = Label(root, text="", font=("Arial", 15, "bold"), bg='#0f6e6e', fg='white')

last_frame_time = time.time()

# ---------------- Learner Mode Variables ----------------
last_learner_message_time = 0
learner_message_interval = 3  # seconds between spoken messages
learner_info_label = Label(root, text="", bg='#c5ece9', fg='black', font=("Arial", 10), wraplength=280, justify=CENTER)

# ---------------- Lane Warning Label (for departure warnings) ----------------
lane_warning_label = Label(root, text="", bg='#c5ece9', fg='black', font=("Arial", 10), wraplength=280, justify=CENTER)

# ---------------- Camera Functions ----------------
def toggle_camera():
    global camera_running, vid
    if not camera_running:
        vid = cv2.VideoCapture(0)  # change the camera if necessary
        camera_running = True
        camera_button.config(text="Stop Camera")
        speak_message("Camera module deployed")
        update_camera()
    else:
        camera_running = False
        camera_button.config(text="Start Camera")
        speak_message("Camera module disabled")
        if vid is not None:
            vid.release()

def update_camera():
    global vid, camera_running, last_frame_time, last_learner_message_time
    global person_warning_count, traffic_warning_count, traffic_light_warning_count
    lane_warning = None  # Initialize lane departure warning message
    if not camera_running:
        return
    ret, frame = vid.read()
    if ret:
        current_time = time.time()
        dt = current_time - last_frame_time
        fps = 1.0 / dt if dt > 0 else 0.0
        last_frame_time = current_time
        if is_developer_mode_active:
            fps_label.config(text=f"FPS: {fps:.2f}")
        frame = cv2.resize(frame, (288, 216))
        
        # Process based on active mode:
        if is_driver_assistance_active:
            left_fit, right_fit, current_mode, lane_frame = detect_lanes(frame, lane_smoother, mode_switcher)
            lane_warning = lane_departure_warning(left_fit, right_fit, frame.shape[1], y_eval=frame.shape[0])
            combined_frame, obj_counts = detect_boxes(lane_frame)
            if lane_warning:
                cv2.putText(combined_frame, lane_warning, (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                speak_message(lane_warning)
            frame = combined_frame
            t = time.time()
            dev_times.append(t)
            lane_ind = 1 if (left_fit is not None and right_fit is not None) else 0
            dev_lane_indicator.append(lane_ind)
            dev_person_counts.append(obj_counts.get(0, 0))
            dev_car_counts.append(obj_counts.get(2, 0))
            dev_motorcycle_counts.append(obj_counts.get(3, 0))
        elif is_lane_detection_active:
            left_fit, right_fit, current_mode, lane_frame = detect_lanes(frame, lane_smoother, mode_switcher)
            lane_warning = lane_departure_warning(left_fit, right_fit, frame.shape[1], y_eval=frame.shape[0])
            if lane_warning:
                cv2.putText(lane_frame, lane_warning, (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)
                speak_message(lane_warning)
            frame = lane_frame
            t = time.time()
            dev_times.append(t)
            lane_ind = 1 if (left_fit is not None and right_fit is not None) else 0
            dev_lane_indicator.append(lane_ind)
            dev_person_counts.append(0)
            dev_car_counts.append(0)
            dev_motorcycle_counts.append(0)
        elif is_box_detection_active:
            frame, obj_counts = detect_boxes(frame)
            t = time.time()
            dev_times.append(t)
            dev_lane_indicator.append(0)
            dev_person_counts.append(obj_counts.get(0, 0))
            dev_car_counts.append(obj_counts.get(2, 0))
            dev_motorcycle_counts.append(obj_counts.get(3, 0))
        elif is_learner_mode_active:
            frame, obj_counts = detect_boxes(frame)
            message = ""
            speak_messages = []
            if obj_counts.get(0, 0) > 5:
                message += "Proceed with caution. Keep your vehicle at slow speeds. Crowd ahead of vehicle. "
                if person_warning_count < 3:
                    speak_messages.append("Proceed with caution. Keep your vehicle at slow speeds. Crowd ahead of vehicle.")
                    person_warning_count += 1
            total_vehicle = obj_counts.get(2, 0) + obj_counts.get(3, 0) + obj_counts.get(7, 0)
            if total_vehicle > 5:
                message += "Keep your vehicle at steady speeds. Traffic ahead of vehicle. "
                if traffic_warning_count < 3:
                    speak_messages.append("Keep your vehicle at steady speeds. Traffic ahead of vehicle.")
                    traffic_warning_count += 1
            if obj_counts.get(9, 0) >= 1:
                message += "Traffic light ahead. Check the signal and respond accordingly."
                if traffic_light_warning_count < 3:
                    speak_messages.append("Traffic light ahead. Check the signal and respond accordingly.")
                    traffic_light_warning_count += 1
            learner_info_label.config(text=message)
            if speak_messages and (current_time - last_learner_message_time > learner_message_interval):
                combined_message = " ".join(speak_messages)
                speak_message(combined_message)
                last_learner_message_time = current_time

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame_pil = Image.fromarray(frame)
        frame_tk = ImageTk.PhotoImage(frame_pil)
        camera_module_label.configure(image=frame_tk)
        camera_module_label.image = frame_tk
        limit_dev_data(50)
        if is_developer_mode_active:
            update_developer_plots()
        
        # Display lane departure warning in a dedicated box if driver assistance or lane detection mode is active.
        if is_driver_assistance_active or is_lane_detection_active:
            lane_warning_label.place(x=703, y=60, width=280, height=30)
            lane_warning_label.config(text=lane_warning if lane_warning is not None else "")
        else:
            lane_warning_label.place_forget()
            
    camera_module_label.after(10, update_camera)

def update_map():
    current_lat = 17.4065
    current_lng = 78.4772
    map_widget.set_position(current_lat, current_lng)
    root.after(2000, update_map)

update_map()

# ---------------- UI Button Toggle Functions ----------------
def toggle_lane_assistance():
    global is_lane_detection_active
    is_lane_detection_active = not is_lane_detection_active
    if is_lane_detection_active:
        lane_button.config(text="Deactivate Lane Assistance")
        disable_driver_assistance()
        disable_object_detection()
        speak_message("lane assistance activated")
    else:
        lane_button.config(text="Activate Lane Assistance")
        speak_message("lane assistance deactivated")

def toggle_object_detection():
    global is_box_detection_active
    is_box_detection_active = not is_box_detection_active
    if is_box_detection_active:
        object_detection_button.config(text="Deactivate Object Detection")
        disable_driver_assistance()
        disable_lane_assistance()
        speak_message("object detection activated")
    else:
        object_detection_button.config(text="Activate Object Detection")
        speak_message("object detection deactivated")

def toggle_driver_assistance():
    global is_driver_assistance_active
    is_driver_assistance_active = not is_driver_assistance_active
    if is_driver_assistance_active:
        driver_assistance_button.config(text="Deactivate Driver Assistance")
        lane_button.config(state=DISABLED)
        object_detection_button.config(state=DISABLED)
        disable_lane_assistance()
        disable_object_detection()
        speak_message("driver assistance activated")
    else:
        driver_assistance_button.config(text="Activate Driver Assistance")
        lane_button.config(state=NORMAL)
        object_detection_button.config(state=NORMAL)
        speak_message("driver assistance deactivated")

def toggle_developer_mode():
    global is_developer_mode_active, dev_frame_left, dev_frame_right
    is_developer_mode_active = not is_developer_mode_active
    if is_developer_mode_active:
        developer_mode_button.config(text="Stop Developer Mode")
        dev_frame_left.place(x=15, y=100, width=380, height=380)
        dev_frame_right.place(x=405 + 288 + 13, y=100, width=380, height=380)
        mode_info_frame.place(x=403, y=330, width=295, height=150)
        fps_label.place(x=990, y=60)
        speak_message("developer mode activated")
    else:
        developer_mode_button.config(text="Activate Developer Mode")
        dev_frame_left.place_forget()
        dev_frame_right.place_forget()
        mode_info_frame.place_forget()
        fps_label.place_forget()
        speak_message("developer mode deactivated")

def toggle_learner_mode():
    global is_learner_mode_active
    is_learner_mode_active = not is_learner_mode_active
    if is_learner_mode_active:
        learner_mode_button.config(text="Deactivate Learner Mode")
        disable_driver_assistance()
        disable_lane_assistance()
        disable_object_detection()
        learner_info_label.place(x=403, y=60, width=294, height=30)
        speak_message("Learner mode activated")
    else:
        learner_mode_button.config(text="Activate Learner Mode")
        learner_info_label.place_forget()
        speak_message("Learner mode deactivated")

def disable_lane_assistance():
    global is_lane_detection_active
    is_lane_detection_active = False
    lane_button.config(text="Activate Lane Assistance")

def disable_object_detection():
    global is_box_detection_active
    is_box_detection_active = False
    object_detection_button.config(text="Activate Object Detection")

def disable_driver_assistance():
    global is_driver_assistance_active
    is_driver_assistance_active = False
    driver_assistance_button.config(text="Activate Driver Assistance")
    lane_button.config(state=NORMAL)
    object_detection_button.config(state=NORMAL)

# ---------------- UI Buttons ----------------
camera_button = Button(root, text="Start Camera", font="Elephant 10", bg='#4da89f',
                       activebackground='#1f534f', fg='#202930', border=2, command=toggle_camera)
camera_button.place(x=504, y=550)

lane_button = Button(root, text="Activate Lane Assistance", font="Elephant 10", bg='#4da89f',
                     activebackground='#1f534f', fg='#202930', border=2, command=toggle_lane_assistance)
lane_button.place(x=70, y=500)

object_detection_button = Button(root, text="Activate Object Detection", font="Elephant 10", bg='#4da89f',
                                 activebackground='#1f534f', fg='#202930', border=2, command=toggle_object_detection)
object_detection_button.place(x=270, y=500)

driver_assistance_button = Button(root, text="Activate Driver Assistance", font="Elephant 10", bg='#4da89f',
                                  activebackground='#1f534f', fg='#202930', border=2, command=toggle_driver_assistance)
driver_assistance_button.place(x=470, y=500)

developer_mode_button = Button(root, text="Activate Developer Mode", font="Elephant 10", bg='#4da89f',
                               activebackground='#1f534f', fg='#202930', border=2, command=toggle_developer_mode)
developer_mode_button.place(x=670, y=500)

learner_mode_button = Button(root, text="Activate Learner Mode", font="Elephant 10", bg='#4da89f',
                             activebackground='#1f534f', fg='#202930', border=2, command=toggle_learner_mode)
learner_mode_button.place(x=870, y=500)

update_mode_info()

# Speak the startup message
speak_message("Advanced Self Driving Online and Ready")

root.mainloop()


In [ ]:
from tkinter import *
from PIL import Image, ImageTk
import cv2
import numpy as np
import time
import warnings
import os, csv, datetime
import tkintermapview
from ultralytics import YOLO
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from sklearn.linear_model import RANSACRegressor  # Added for robust fitting
import pyttsx3  # Text-to-speech

# Initialize the TTS engine
tts_engine = pyttsx3.init()
tts_engine.setProperty('rate', 150)  # Slower speech rate
voices = tts_engine.getProperty('voices')
for idx, voice in enumerate(voices):
    print(f"Voice {idx}: {voice.name}")
tts_engine.setProperty('voice', voices[0].id)  # Choose a voice
tts_engine.setProperty('volume', 0.9)

def speak_message(message):
    tts_engine.say(message)
    tts_engine.runAndWait()


# --------------------- Dataset Recording Setup ---------------------
record_data_mode = False
save_interval = 1.0  # seconds between captures
last_save_time = 0
output_dir = "./recorded_dataset"
csv_path = os.path.join(output_dir, "metadata.csv")

os.makedirs(output_dir, exist_ok=True)
if not os.path.isfile(csv_path):
    with open(csv_path, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["timestamp", "filename", "mode", "left_fit", "right_fit", "obj_counts"])

# --------------------- Enhanced Lane Detection Classes ---------------------
class LaneSmoother:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.left_fit = None
        self.right_fit = None

    def smooth(self, left_fit, right_fit):
        if left_fit is None or right_fit is None:
            self.left_fit, self.right_fit = left_fit, right_fit
        else:
            if self.left_fit is None:
                self.left_fit = left_fit
            else:
                self.left_fit = self.alpha * left_fit + (1 - self.alpha) * self.left_fit
            if self.right_fit is None:
                self.right_fit = right_fit
            else:
                self.right_fit = self.alpha * right_fit + (1 - self.alpha) * self.right_fit
        return self.left_fit, self.right_fit

class ModeSwitcher:
    def __init__(self, curvature_threshold=0.0003, straight_time=2.0, curve_time=1.0):
        self.mode = "curved"
        self.curvature_threshold = curvature_threshold
        self.straight_time = straight_time
        self.curve_time = curve_time
        self.straight_start = None
        self.curve_start = None

    def update(self, left_fit, right_fit):
        now = time.time()
        coeffs = []
        if left_fit is not None:
            coeffs.append(abs(left_fit[0]))
        if right_fit is not None:
            coeffs.append(abs(right_fit[0]))
        if not coeffs:
            return self.mode
        avg_curv = np.mean(coeffs)
        if self.mode == "curved":
            if avg_curv < self.curvature_threshold:
                if self.straight_start is None:
                    self.straight_start = now
                elif now - self.straight_start >= self.straight_time:
                    self.mode = "straight"
                    self.straight_start = None
            else:
                self.straight_start = None
        else:
            if avg_curv >= self.curvature_threshold:
                if self.curve_start is None:
                    self.curve_start = now
                elif now - self.curve_start >= self.curve_time:
                    self.mode = "curved"
                    self.curve_start = None
            else:
                self.curve_start = None
        return self.mode

# --------------------- Perspective & Sliding Window ---------------------
def undistort_image(image):
    return image

def get_perspective_transform(image):
    h, w = image.shape[:2]
    src = np.float32([[w*0.45,h*0.65],[w*0.55,h*0.65],[w*0.9,h],[w*0.1,h]])
    dst = np.float32([[w*0.2,0],[w*0.8,0],[w*0.8,h],[w*0.2,h]])
    M  = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    return M, Minv

def warp_image(image, M):
    return cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]), flags=cv2.INTER_LINEAR)

def preprocess_lane_image(image):
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS)
    white_mask  = cv2.inRange(hls, np.array([0,200,0]), np.array([255,255,255]))
    yellow_mask = cv2.inRange(hls, np.array([15,30,115]), np.array([35,204,255]))
    mask = cv2.bitwise_or(white_mask, yellow_mask)
    res = cv2.bitwise_and(image, image, mask=mask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    _, binary = cv2.threshold(blur, 160, 255, cv2.THRESH_BINARY)
    return binary

def sliding_window_lane_search(binary_warped):
    hist = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    midpoint = hist.shape[0]//2
    leftx_base  = np.argmax(hist[:midpoint])
    rightx_base = np.argmax(hist[midpoint:]) + midpoint

    nwindows = 9
    window_h = binary_warped.shape[0]//nwindows
    nonzero = binary_warped.nonzero()
    nz_y, nz_x = nonzero
    margin, minpix = 100, 50
    left_current, right_current = leftx_base, rightx_base
    left_inds, right_inds = [], []

    for win in range(nwindows):
        ylow = binary_warped.shape[0] - (win+1)*window_h
        yhigh= binary_warped.shape[0] - win*window_h
        xl, xh = left_current-margin, left_current+margin
        rl, rh = right_current-margin, right_current+margin

        gL = ((nz_y>=ylow)&(nz_y<yhigh)&(nz_x>=xl)&(nz_x<xh)).nonzero()[0]
        gR = ((nz_y>=ylow)&(nz_y<yhigh)&(nz_x>=rl)&(nz_x<rh)).nonzero()[0]
        left_inds.append(gL); right_inds.append(gR)

        if len(gL)>minpix: left_current = int(np.mean(nz_x[gL]))
        if len(gR)>minpix: right_current= int(np.mean(nz_x[gR]))

    left_inds  = np.concatenate(left_inds)
    right_inds = np.concatenate(right_inds)

    left_fit  = np.polyfit(nz_y[left_inds],  nz_x[left_inds],  2) if len(left_inds)>0  else None
    right_fit = np.polyfit(nz_y[right_inds], nz_x[right_inds], 2) if len(right_inds)>0 else None

    return left_fit, right_fit, np.dstack((binary_warped,)*3)

def draw_lane_overlay(orig, binary_warped, left_fit, right_fit, Minv):
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0])
    warp_zero = np.zeros_like(orig).astype(np.uint8)
    if left_fit is not None and right_fit is not None:
        lx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        rx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        pts = np.hstack((np.vstack((lx,ploty)).T, np.flipud(np.vstack((rx,ploty)).T)))
        cv2.fillPoly(warp_zero, np.int_([pts]), (0,255,0))
    newwarp = cv2.warpPerspective(warp_zero, Minv, (orig.shape[1], orig.shape[0]))
    return cv2.addWeighted(orig,1,newwarp,0.3,0)

def detect_lanes(frame, smoother, mode_switcher):
    undist = undistort_image(frame)
    M, Minv = get_perspective_transform(undist)
    warped = warp_image(undist, M)
    binary = preprocess_lane_image(warped)
    left_fit, right_fit, _ = sliding_window_lane_search(binary)
    mode_switcher.update(left_fit, right_fit)
    if left_fit is not None and right_fit is not None:
        left_fit, right_fit = smoother.smooth(left_fit, right_fit)
    out = draw_lane_overlay(frame, binary, left_fit, right_fit, Minv)
    return left_fit, right_fit, out

# ---------------- lane departure warning ---------------------
lane_warning_spoken = False
def lane_departure_warning(left_fit, right_fit, fw, y):
    global lane_warning_spoken
    thr = 50
    if left_fit is None or right_fit is None:
        lane_warning_spoken=False; return None
    lx = np.polyval(left_fit, y)
    rx = np.polyval(right_fit, y)
    center = (lx+rx)/2; off = fw/2 - center
    if abs(off)>thr and not lane_warning_spoken:
        lane_warning_spoken=True
        d = 'left' if off<0 else 'right'
        return f"Lane departure warning! Your vehicle is turning {d}. Regain balance."
    if abs(off)<=thr: lane_warning_spoken=False
    return None

# ---------------- object detection ---------------------
yolo_model = YOLO("yolov8n.pt")
dev_log_messages = []

def detect_boxes(frame):
    global dev_log_messages
    results = yolo_model(frame)
    counts = {}
    for res in results:
        for b in res.boxes:
            c = int(b.cls[0]); counts[c]=counts.get(c,0)+1
            x1,y1,x2,y2 = map(int,b.xyxy[0])
            cv2.rectangle(frame,(x1,y1),(x2,y2),(255,0,0),2)
            lbl = yolo_model.names.get(c,str(c))
            cv2.putText(frame,lbl,(x1,y1-10),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)
    if is_developer_mode_active:
        log = f"{frame.shape[0]}x{frame.shape[1]} "+", ".join(f"{v} {yolo_model.names.get(k,'')}" for k,v in counts.items())
        dev_log_messages.append(log); dev_log_messages=dev_log_messages[-10:]
        update_dev_log_widget()
    return frame, counts

# ---------------- developer plotting & map ---------------------
dev_times=[]; dev_lane_indicator=[]; dev_person_counts=[]; dev_car_counts=[]; dev_motorcycle_counts=[]
def update_developer_plots():
    dev_fig.clf()
    ax1=dev_fig.add_subplot(211); ax2=dev_fig.add_subplot(212)
    if dev_times:
        ax1.plot(dev_times, dev_lane_indicator, label="Lane")
        ax1.set_ylim(-0.1,1.1); ax1.legend(fontsize=8)
        ax2.plot(dev_times, dev_person_counts, label="Persons")
        ax2.plot(dev_times, dev_car_counts, label="Cars")
        ax2.plot(dev_times, dev_motorcycle_counts, label="Motorcycles")
        ax2.legend(fontsize=8)
    dev_fig.tight_layout(); dev_canvas.draw()

def update_dev_log_widget():
    dev_log_widget.config(state=NORMAL)
    dev_log_widget.delete("1.0",END)
    dev_log_widget.insert(END,"\n".join(dev_log_messages))
    dev_log_widget.config(state=DISABLED)

def update_map():
    map_widget.set_position(17.4065,78.4772)
    root.after(2000, update_map)

# ---------------- flags & GUI setup ---------------------
vid=None; camera_running=False
is_lane_detection_active=False; is_box_detection_active=False
is_driver_assistance_active=False; is_developer_mode_active=False; is_learner_mode_active=False
person_warning_count=0; traffic_warning_count=0; traffic_light_warning_count=0

root=Tk(); root.title("Advanced Self Driving"); root.geometry("1100x600"); root.resizable(0,0)
bg=PhotoImage(file="background.png"); Label(root,image=bg).place(x=0,y=0,relwidth=1,relheight=1)
Label(root,text="Advanced\nSelf Driving",font="Elephant 18 bold",bg='#0f6e6e',fg='white').place(x=90,y=30)
root.iconphoto(False,PhotoImage(file="road.png"))
logo=PhotoImage(file="smart-car.png").subsample(8,8); Label(root,image=logo,bg='#0f6e6e').place(x=253,y=30)
cam_mod=PhotoImage(file="camera_module.png").subsample(2,2)
camera_module_label=Label(root,image=cam_mod,bg='#c5ece9',bd=3); camera_module_label.place(x=403,y=100)

dev_frame_left=Frame(root,bg='#c5ece9',bd=2,relief=SUNKEN)
dev_frame_right=Frame(root,bg='#c5ece9',bd=2,relief=SUNKEN)
dev_frame_left.place_forget(); dev_frame_right.place_forget()
dev_fig=plt.Figure(figsize=(3,2),dpi=80); dev_canvas=FigureCanvasTkAgg(dev_fig,master=dev_frame_left)
dev_canvas.get_tk_widget().pack(fill=BOTH,expand=True)
map_widget=tkintermapview.TkinterMapView(dev_frame_right,corner_radius=0); map_widget.set_zoom(10)
map_widget.grid(row=0,column=0,sticky="nsew")
dev_log_widget=Text(dev_frame_right,state=DISABLED,font=("Consolas",8))
dev_log_widget.grid(row=1,column=0,sticky="nsew")

mode_info_frame=Frame(root,bg='#c5ece9',bd=2,relief=SUNKEN)
mode_info_label=Label(mode_info_frame,text="",font=("Arial",10),bg='#c5ece9',wraplength=280,justify=CENTER)
mode_info_label.pack(fill=BOTH,expand=True)
fps_label=Label(root,text="",font=("Arial",15,"bold"),bg='#0f6e6e',fg='white')

learner_info_label=Label(root,text="",bg='#c5ece9',fg='black',font=("Arial",10),wraplength=280,justify=CENTER)
lane_warning_label = Label(root,text="",bg='#c5ece9',fg='black',font=("Arial",10),wraplength=280,justify=CENTER)

# ---------------- camera & update loop ---------------------
def toggle_camera():
    global camera_running, vid
    if not camera_running:
        vid=cv2.VideoCapture(0); camera_running=True
        camera_button.config(text="Stop Camera"); speak_message("Camera module deployed")
        update_camera()
    else:
        camera_running=False; camera_button.config(text="Start Camera")
        speak_message("Camera module disabled"); vid.release()

def update_camera():
    global last_frame_time, last_save_time, person_warning_count, traffic_warning_count, traffic_light_warning_count
    if not camera_running: return
    ret, frame=vid.read()
    if not ret: return
    now=time.time()
    if 'last_frame_time' in globals():
        fps=1.0/(now-last_frame_time) if now!=last_frame_time else 0
    else:
        fps=0
    last_frame_time=now
    if is_developer_mode_active: fps_label.config(text=f"FPS: {fps:.2f}")
    frame=cv2.resize(frame,(288,216))
    obj_counts={}; lfit=rfit=None

    if is_driver_assistance_active:
        lfit,rfit,frame=detect_lanes(frame, lane_smoother, mode_switcher)
        warn=lane_departure_warning(lfit,rfit,frame.shape[1],frame.shape[0])
        frame,obj_counts=detect_boxes(frame)
        if warn: cv2.putText(frame,warn,(50,70),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3); speak_message(warn)
    elif is_lane_detection_active:
        lfit,rfit,frame=detect_lanes(frame, lane_smoother, mode_switcher)
        warn=lane_departure_warning(lfit,rfit,frame.shape[1],frame.shape[0])
        if warn: cv2.putText(frame,warn,(50,70),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3); speak_message(warn)
    elif is_box_detection_active:
        frame,obj_counts=detect_boxes(frame)
    elif is_learner_mode_active:
        frame,obj_counts=detect_boxes(frame)
        # learner warnings logic here...

    # recording
    if record_data_mode and (now - last_save_time >= save_interval):
        ts = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
        fname = f"frame_{ts}.png"; path=os.path.join(output_dir,fname)
        cv2.imwrite(path, cv2.cvtColor(frame,cv2.COLOR_BGR2RGB))
        with open(csv_path,'a',newline='') as f:
            writer=csv.writer(f)
            writer.writerow([ts, fname,
                ('driver' if is_driver_assistance_active else
                 'lane'   if is_lane_detection_active   else
                 'object' if is_box_detection_active    else
                 'learner' if is_learner_mode_active    else 'raw'),
                (lfit.tolist() if lfit is not None else None),
                (rfit.tolist() if rfit is not None else None),
                obj_counts])
        last_save_time=now

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    imgtk = ImageTk.PhotoImage(Image.fromarray(frame_rgb))
    camera_module_label.config(image=imgtk); camera_module_label.image=imgtk

    if is_developer_mode_active: update_developer_plots()
    camera_module_label.after(10, update_camera)

# ---------------- toggles & UI buttons ---------------------
def toggle_lane_assistance():
    global is_lane_detection_active
    is_lane_detection_active = not is_lane_detection_active
    lane_button.config(text="Deactivate Lane Assistance" if is_lane_detection_active else "Activate Lane Assistance")
    if is_lane_detection_active:
        disable_driver_assistance(); disable_object_detection(); speak_message("lane assistance activated")
    else:
        speak_message("lane assistance deactivated")

def toggle_object_detection():
    global is_box_detection_active
    is_box_detection_active = not is_box_detection_active
    object_detection_button.config(text="Deactivate Object Detection" if is_box_detection_active else "Activate Object Detection")
    if is_box_detection_active:
        disable_driver_assistance(); disable_lane_assistance(); speak_message("object detection activated")
    else:
        speak_message("object detection deactivated")

def toggle_driver_assistance():
    global is_driver_assistance_active
    is_driver_assistance_active = not is_driver_assistance_active
    driver_assistance_button.config(text="Deactivate Driver Assistance" if is_driver_assistance_active else "Activate Driver Assistance")
    if is_driver_assistance_active:
        lane_button.config(state=DISABLED); object_detection_button.config(state=DISABLED)
        disable_lane_assistance(); disable_object_detection(); speak_message("driver assistance activated")
    else:
        lane_button.config(state=NORMAL); object_detection_button.config(state=NORMAL); speak_message("driver assistance deactivated")

def toggle_developer_mode():
    global is_developer_mode_active
    is_developer_mode_active = not is_developer_mode_active
    developer_mode_button.config(text="Stop Developer Mode" if is_developer_mode_active else "Activate Developer Mode")
    if is_developer_mode_active:
        dev_frame_left.place(x=15,y=100,width=380,height=380)
        dev_frame_right.place(x=705,y=100,width=380,height=380)
        mode_info_frame.place(x=403,y=330,width=295,height=150)
        fps_label.place(x=990,y=60); speak_message("developer mode activated")
    else:
        dev_frame_left.place_forget(); dev_frame_right.place_forget()
        mode_info_frame.place_forget(); fps_label.place_forget()
        speak_message("developer mode deactivated")

def toggle_learner_mode():
    global is_learner_mode_active
    is_learner_mode_active = not is_learner_mode_active
    learner_mode_button.config(text="Deactivate Learner Mode" if is_learner_mode_active else "Activate Learner Mode")
    if is_learner_mode_active:
        disable_driver_assistance(); disable_lane_assistance(); disable_object_detection()
        learner_info_label.place(x=403,y=60,width=294,height=30); speak_message("Learner mode activated")
    else:
        learner_info_label.place_forget(); speak_message("Learner mode deactivated")

def toggle_record_mode():
    global record_data_mode, last_save_time
    record_data_mode = not record_data_mode
    last_save_time = time.time()
    record_button.config(text="Stop Recording" if record_data_mode else "Start Recording")

def disable_lane_assistance():
    global is_lane_detection_active
    is_lane_detection_active=False; lane_button.config(text="Activate Lane Assistance")

def disable_object_detection():
    global is_box_detection_active
    is_box_detection_active=False; object_detection_button.config(text="Activate Object Detection")

def disable_driver_assistance():
    global is_driver_assistance_active
    is_driver_assistance_active=False
    driver_assistance_button.config(text="Activate Driver Assistance")
    lane_button.config(state=NORMAL); object_detection_button.config(state=NORMAL)

def update_mode_info():
    info=""
    if is_lane_detection_active: info+="Lane detection ON\n"
    if is_box_detection_active: info+="Object detection ON\n"
    if is_driver_assistance_active: info+="Driver assistance ON\n"
    info+=f"\nStatus: Lane {'ON' if is_lane_detection_active else 'OFF'} | Obj {'ON' if is_box_detection_active else 'OFF'} | Driver {'ON' if is_driver_assistance_active else 'OFF'}"
    mode_info_label.config(text=info)
    root.after(500, update_mode_info)

camera_button = Button(root, text="Start Camera", font="Elephant 10", bg='#4da89f', fg='#202930', command=toggle_camera)
camera_button.place(x=555,y=550)
lane_button   = Button(root, text="Activate Lane Assistance", font="Elephant 10", bg='#4da89f', fg='#202930', command=toggle_lane_assistance)
lane_button.place(x=70,y=500)
object_detection_button = Button(root, text="Activate Object Detection", font="Elephant 10", bg='#4da89f', fg='#202930', command=toggle_object_detection)
object_detection_button.place(x=270,y=500)
driver_assistance_button = Button(root, text="Activate Driver Assistance", font="Elephant 10", bg='#4da89f', fg='#202930', command=toggle_driver_assistance)
driver_assistance_button.place(x=470,y=500)
developer_mode_button = Button(root, text="Activate Developer Mode", font="Elephant 10", bg='#4da89f', fg='#202930', command=toggle_developer_mode)
developer_mode_button.place(x=670,y=500)
learner_mode_button  = Button(root, text="Activate Learner Mode", font="Elephant 10", bg='#4da89f', fg='#202930', command=toggle_learner_mode)
learner_mode_button.place(x=870,y=500)
record_button = Button(root, text="Start Recording", font="Elephant 10", bg='#ffa500', fg='#202930', command=toggle_record_mode)
record_button.place(x=430,y=550)

update_map()
update_mode_info()
speak_message("Advanced Self Driving Online and Ready")
root.mainloop()


In [ ]:
import pandas as pd
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load & parse
df = pd.read_csv("recorded_dataset/metadata.csv")
df['left_fit']  = df['left_fit'].apply(lambda s: ast.literal_eval(s) if pd.notnull(s) else None)
df['right_fit'] = df['right_fit'].apply(lambda s: ast.literal_eval(s) if pd.notnull(s) else None)

# (Assume you also have ground‐truth columns 'true_left_fit' in same format)
df['true_left_fit'] = df['true_left_fit'].apply(lambda s: ast.literal_eval(s) if pd.notnull(s) else None)

# 2. Compute bottom‐of‐frame lateral offset error
def bottom_offset_error(pred, true, y=216):
    if pred is None or true is None: 
        return np.nan
    x_pred = np.polyval(pred, y)
    x_true = np.polyval(true, y)
    return abs(x_pred - x_true)

df['offset_err_px'] = df.apply(
    lambda row: bottom_offset_error(row['left_fit'], row['true_left_fit']), axis=1
)

# 3. Plot histogram of offset errors with seaborn
plt.figure(figsize=(8,4))
sns.histplot(df['offset_err_px'].dropna(), bins=30, kde=True)
plt.title("Lane‐Center Offset Error at Bottom of Frame")
plt.xlabel("Absolute Offset Error (pixels)")
plt.ylabel("Frame Count")
plt.tight_layout()
plt.show()

# 4. Time‐series of object‐counts (e.g. persons)
# assume df has a datetime index or timestamp column we convert
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y%m%d_%H%M%S")
df = df.set_index('timestamp').sort_index()

plt.figure(figsize=(10,3))
sns.lineplot(data=df, x=df.index, y='obj_counts', estimator=None)
plt.title("Total Object Count Over Time")
plt.xlabel("Time")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 5. Violin plot of per‐mode error distributions
plt.figure(figsize=(6,4))
sns.violinplot(data=df, x='mode', y='offset_err_px', inner="quartile")
plt.title("Offset Error by Recording Mode")
plt.xlabel("Mode")
plt.ylabel("Offset Error (px)")
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import ast
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Load & parse
df = pd.read_csv("recorded_dataset/metadata.csv")
df['left_fit']  = df['left_fit'].apply(lambda s: ast.literal_eval(s) if pd.notnull(s) else None)
df['right_fit'] = df['right_fit'].apply(lambda s: ast.literal_eval(s) if pd.notnull(s) else None)
df['obj_counts'] = df['obj_counts'].apply(lambda s: ast.literal_eval(s) if pd.notnull(s) else {})

# 2. Expand obj_counts → separate columns
counts_df = df['obj_counts'].apply(pd.Series).fillna(0).astype(int)
counts_df.columns = [f"count_{cls}" for cls in counts_df.columns]
df = pd.concat([df, counts_df], axis=1)

# 3. Extract polynomial coeffs into their own columns
for side in ['left', 'right']:
    for i in range(3):
        df[f"{side}_c{i}"] = df[f"{side}_fit"].apply(
            lambda arr: arr[i] if isinstance(arr, (list, tuple, np.ndarray)) else np.nan
        )

# 4. Compute frame-to-frame diffs (stability)
for side in ['left', 'right']:
    for i in range(3):
        df[f"{side}_c{i}_diff"] = df[f"{side}_c{i}"].diff().abs()

# 5. Timestamp → datetime index
df['timestamp'] = pd.to_datetime(df['timestamp'], format="%Y%m%d_%H%M%S")
df = df.set_index('timestamp').sort_index()

# 6. Plot coefficient-diff histograms
plt.figure(figsize=(8,4))
sns.histplot(df['left_c0_diff'].dropna(), bins=30, kde=True)
plt.title("Frame-to-Frame |Δ left_fit[0]| Distribution")
plt.xlabel("Absolute Δ (poly coeff)")
plt.ylabel("Frame Count")
plt.tight_layout()
plt.show()

# 7. Plot object-counts over time (e.g. persons)
plt.figure(figsize=(10,3))
sns.lineplot(data=df, x=df.index, y='count_0')  # replace 'count_0' with the class index you care about
plt.title("Person Count Over Time")
plt.xlabel("Time")
plt.ylabel("Count")
plt.tight_layout()
plt.show()

# 8. Violin of stability by mode
plt.figure(figsize=(6,4))
sns.violinplot(
    data=df.reset_index(),
    x='mode',
    y='left_c0_diff',
    inner="quartile"
)
plt.title("Stability of left_fit[0] by Mode")
plt.xlabel("Mode")
plt.ylabel("|Δ coeff|")
plt.tight_layout()
plt.show()
